# The code for the "song recommender" app

In [1]:
# import relevant libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re
import random
import config

In [2]:
top_100 = pd.read_csv("top_100.csv", index_col=False)

In [3]:
top_100.head()

,Unnamed: 0,Artist,Song
0,0,Miley Cyrus,Flowers
1,1,SZA,Kill Bill
2,2,Morgan Wallen,Last Night
3,3,Sam Smith & Kim Petras,Unholy
4,4,"Metro Boomin, The Weeknd & 21 Savage",Creepin'


In [4]:
# writing the song recommender function
def song_recommender():
    """Function that prompts the user for song input
    and checks whether it's in the top-100 csv file."""
    top_100 = pd.read_csv("top_100.csv")
    user_input = input("Please type in your favorite song: ")
    if user_input.lower() in map(str.lower, list(top_100["Song"])):
        print("Your song is in the current top-100.")
        random_song = random.choice(list(top_100["Song"]))
        artist = top_100.loc[top_100["Song"] == random_song, 'Artist'].to_string(index=False)
        print(f"\nHere is another song from the top-100 you might like: {random_song} by {artist}")
    else: 
        print("Your song is not in the current top-100. Let me give you a suggestion for another song.")

In [5]:
# writing the while loop that ensures the programme keeps running
while True:
    song_recommender()
    user_input = input("\nDo you want to input another song? yes/no: ")
    if user_input == "yes":
        continue
    elif user_input == "no":
        print("Your session has ended. Thank you for using our app!")
        break
    else:
        print("Please write either 'yes' or 'no'")
        continue

Please type in your favorite song: Flowers
Your song is in the current top-100.

Here is another song from the top-100 you might like: Tennessee Fan by Morgan Wallen

Do you want to input another song? yes/no: no
Your session has ended. Thank you for using our app!


In [6]:
# Note: need to write a function that processes the song provided by the user and recommends another song based on that.

In [7]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!")

# Creating the song database with audio features

In [8]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


# initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [9]:
# checking a playlist
playlist = sp.user_playlist_tracks("spotify", "27gN69ebwiJRtXEboL12Ih")
playlist["items"][0]

{'added_at': '2022-11-26T02:31:48Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/celsum76'},
  'href': 'https://api.spotify.com/v1/users/celsum76',
  'id': 'celsum76',
  'type': 'user',
  'uri': 'spotify:user:celsum76'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5M52tdBnJaKSvOpJGz8mfZ'},
     'href': 'https://api.spotify.com/v1/artists/5M52tdBnJaKSvOpJGz8mfZ',
     'id': '5M52tdBnJaKSvOpJGz8mfZ',
     'name': 'Black Sabbath',
     'type': 'artist',
     'uri': 'spotify:artist:5M52tdBnJaKSvOpJGz8mfZ'}],
   'available_markets': [],
   'external_urls': {'spotify': 'https://open.spotify.com/album/1JA2UhLRbFRkmoh6Lz64KH'},
   'href': 'https://api.spotify.com/v1/albums/1JA2UhLRbFRkmoh6Lz64KH',
   'id': '1JA2UhLRbFRkmoh6Lz64KH',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b2738c9131b3b0f5a4b9fdc5a0ac',
    

In [10]:
# function that retrieves the songs from a playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [11]:
# creating a list with playlists id's and defining the variable 'tracks' to store the results
playlist_ids = ["27gN69ebwiJRtXEboL12Ih", "37i9dQZF1DX4pUKG1kS0Ac", "37i9dQZF1DXbITWG1ZJKYt", "37i9dQZF1DXbSbnqxMTGx9", "37i9dQZF1DX8a1tdzq5tbM", "37i9dQZF1EQpj7X7UK8OOF", "37i9dQZF1EQqkOPvHGajmW", "17IFbN8moTMWsaK5S5qCyD"]
tracks = []
# genres included: metal, guilty pleasures, jazz, reggae, dance, rock, indie, rap/hiphop

# for loop that loops through the playlist ids and retrieves the song with the get_playlist_tracks function
for i in playlist_ids:
    tracks += get_playlist_tracks("spotify", i)

In [12]:
# checking how many tracks are in the Pandas dataframe
print(len(tracks))

1043


In [13]:
# getting the audio features
list_of_audio_features = []
for item in range(len(tracks)):
    #print (tracks[item]["track"]["id"])
    list_of_audio_features.append(sp.audio_features(tracks[item]["track"]["id"])[0])

In [14]:
print(len(tracks))

1043


In [15]:
songs_df = pd.DataFrame(list_of_audio_features)    
songs_df = songs_df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

songs_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.423,0.685,-9.651,0.0780,0.000063,0.006420,0.1330,0.328,162.780,3q9G9SvgdfE1fxlvoDIb20,167720
1,0.324,0.991,-4.465,0.1970,0.002470,0.008950,0.1040,0.195,140.245,47FyQCd3TYLrZ9TU6MPaWK,166707
2,0.308,0.866,-5.671,0.0590,0.000739,0.001410,0.2320,0.227,109.825,6p8eFfPw3nQkf37aT3AkmK,438120
3,0.453,0.905,-4.001,0.1150,0.360000,0.000569,0.2600,0.474,137.571,2b9lp5A6CqSzwOrBfAFhof,296200
4,0.420,0.930,-5.583,0.0638,0.018900,0.006750,0.2640,0.344,99.641,1s4Ie0cT6P73SRSfh3oyGW,291373
...,...,...,...,...,...,...,...,...,...,...,...
1038,0.794,0.614,-5.352,0.0467,0.096400,0.000000,0.0788,0.663,80.569,7jLbTp3qZzah9kMIdW8e5M,238053
1039,0.780,0.575,-7.247,0.2730,0.430000,0.000000,0.1430,0.773,84.492,2g8HN35AnVGIk7B8yMucww,252747
1040,0.748,0.614,-6.004,0.4070,0.067200,0.000078,0.1060,0.570,172.030,36UjTTrSwaAqLkAlf2Ooc4,230493
1041,0.731,0.807,-6.492,0.1000,0.181000,0.751000,0.3190,0.765,114.759,2bLqfJjuC5syrsgDsZfGmn,179120


In [16]:
songs_df.to_csv("songs_df.csv", index=False)

In [17]:
# description for Thursday:

# user input a song
## get the audio features for the song from spotify 
## scale the audio features with scaler that you build the Kmean on
## predict for the user song(audio features) in which cluster belong to (e.g belongs to 7 ) 
## got the 100,000000 database and recommed a song that belong to the same cluster of the user input song 

# practical implementation:

# ask user for song input
# retrieve id and audio features through Spotsipy
# Use StandardScaler() on song by user
# Use KMeans.predict() to predict what cluster the song by user belongs to
# Recommend a song in a similar cluster from the song_df

# Scaling the audio features of the song database and requesting the user's song

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
import numpy as np

In [19]:
audio_features = songs_df[["danceability", "energy", "loudness", "speechiness", "acousticness", "liveness", "valence", "tempo"]]

In [20]:
audio_features.head()

,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo
0,0.423,0.685,-9.651,0.0780,0.000063,0.133,0.328,162.780
1,0.324,0.991,-4.465,0.1970,0.002470,0.104,0.195,140.245
2,0.308,0.866,-5.671,0.0590,0.000739,0.232,0.227,109.825
3,0.453,0.905,-4.001,0.1150,0.360000,0.260,0.474,137.571
4,0.420,0.930,-5.583,0.0638,0.018900,0.264,0.344,99.641


In [21]:
scaler = StandardScaler()
scaler.fit(audio_features)
audio_features_scaled = scaler.transform(audio_features)
audio_features_scaled

array([[-1.16461769,  0.33087088, -0.12830809, ..., -0.3174359 ,
        -0.96782921,  1.55682051],
       [-1.74248023,  1.48755614,  0.97919018, ..., -0.51738807,
        -1.48472942,  0.77561523],
       [-1.83587216,  1.01505399,  0.72164238, ...,  0.36515947,
        -1.36036245, -0.2789339 ],
       ...,
       [ 0.73240581,  0.06248966,  0.65052843, ..., -0.50359827,
        -0.02730402,  1.87748387],
       [ 0.63317689,  0.79203298,  0.5463134 , ...,  0.965016  ,
         0.73055719, -0.10789033],
       [ 0.24793519,  1.18137475,  0.93284866, ...,  0.33068495,
         1.20470625, -1.17075937]])

In [22]:
kmeans = KMeans(n_clusters=8, random_state=1234)
kmeans.fit(audio_features_scaled)
# assign a cluster to each example
labels = kmeans.predict(audio_features_scaled)
# retrieve unique clusters
clusters = np.unique(labels)
# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = np.where(labels == cluster)
    # create scatter of these samples
    pyplot.scatter(audio_features.to_numpy()[row_ix, 0], audio_features.to_numpy()[row_ix,1])
    pyplot.xlabel("energy")
    pyplot.ylabel("instrumentalness")
    # show the plot
pyplot.show()

In [23]:
clusters = kmeans.predict(audio_features_scaled)
# clusters
pd.Series(clusters).value_counts().sort_index()

0    122
1    173
2    127
3     72
4    189
5    122
6     80
7    158
dtype: int64

In [24]:
# saving with pickle
import pickle

# scaler = StandardScaler()
# model = KMeans()

with open("scaler_song_recommender", "wb") as f:
    pickle.dump(scaler,f)

with open("kmeans_song_recommender", "wb") as f:
    pickle.dump(kmeans,f)

In [25]:
# getting the scaled songs back into a Pandas dataframe
scaled_songs_df = pd.DataFrame(audio_features_scaled, columns=["danceability", "energy", "loudness", "speechiness", "acousticness", "liveness", "valence", "tempo"])
scaled_songs_df.head()

,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo
0,-1.164618,0.330871,-0.128308,-0.114356,-0.906894,-0.317436,-0.967829,1.556821
1,-1.742480,1.487556,0.979190,1.318250,-0.899905,-0.517388,-1.484729,0.775615
2,-1.835872,1.015054,0.721642,-0.343092,-0.904931,0.365159,-1.360362,-0.278934
3,-0.989508,1.162475,1.078280,0.331076,0.138198,0.558217,-0.400405,0.682918
4,-1.182129,1.256975,0.740435,-0.285306,-0.852200,0.585796,-0.905646,-0.631976


In [26]:
scaled_songs_df

,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo
0,-1.164618,0.330871,-0.128308,-0.114356,-0.906894,-0.317436,-0.967829,1.556821
1,-1.742480,1.487556,0.979190,1.318250,-0.899905,-0.517388,-1.484729,0.775615
2,-1.835872,1.015054,0.721642,-0.343092,-0.904931,0.365159,-1.360362,-0.278934
3,-0.989508,1.162475,1.078280,0.331076,0.138198,0.558217,-0.400405,0.682918
4,-1.182129,1.256975,0.740435,-0.285306,-0.852200,0.585796,-0.905646,-0.631976
...,...,...,...,...,...,...,...,...
1038,1.000908,0.062490,0.789767,-0.491168,-0.627176,-0.691140,0.334137,-1.293131
1039,0.919190,-0.084931,0.385079,2.233191,0.341445,-0.248487,0.761649,-1.157136
1040,0.732406,0.062490,0.650528,3.846378,-0.711959,-0.503598,-0.027304,1.877484
1041,0.633177,0.792033,0.546313,0.150495,-0.381536,0.965016,0.730557,-0.107890


# Scaling the audio features of song provided by the user

In [27]:
# seaching spotify for song provided by the user
# todo: add artist to be more specific
user_song = "Ride the Lightning"
result = sp.search(q=user_song,limit=1)

In [28]:
# getting the id of a song
result["tracks"]["items"][0]["id"]

'1JTQG8J8jN8tjDTyUWnaRe'

In [29]:
# getting the audio features of a song using the song id
user_song_audio_features = sp.audio_features(result["tracks"]["items"][0]["id"])
user_song_audio_features

[{'danceability': 0.576,
  'energy': 0.474,
  'key': 1,
  'loudness': -6.342,
  'mode': 1,
  'speechiness': 0.0331,
  'acousticness': 0.75,
  'instrumentalness': 8.74e-05,
  'liveness': 0.0944,
  'valence': 0.4,
  'tempo': 77.834,
  'type': 'audio_features',
  'id': '1JTQG8J8jN8tjDTyUWnaRe',
  'uri': 'spotify:track:1JTQG8J8jN8tjDTyUWnaRe',
  'track_href': 'https://api.spotify.com/v1/tracks/1JTQG8J8jN8tjDTyUWnaRe',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1JTQG8J8jN8tjDTyUWnaRe',
  'duration_ms': 186459,
  'time_signature': 4}]

In [30]:
# creating a Pandas dataframe with the audio features of the user song
user_song_df = pd.DataFrame(user_song_audio_features)    
user_song_df = user_song_df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

user_song_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.576,0.474,-6.342,0.0331,0.75,0.000087,0.0944,0.4,77.834,1JTQG8J8jN8tjDTyUWnaRe,186459


In [31]:
user_song_audio_features = user_song_df[["danceability", "energy", "loudness", "speechiness", "acousticness", "liveness", "valence", "tempo"]]

In [32]:
# scaling the song provided by the user
user_song_audio_features_scaled = scaler.transform(user_song_audio_features)
user_song_audio_features_scaled

array([[-0.2715574 , -0.46671274,  0.57834671, -0.65489398,  1.27057807,
        -0.58357914, -0.68800354, -1.3879438 ]])

In [35]:
# use kmeans.predict() to predict what cluster the song by user belongs to
label = kmeans.predict(user_song_audio_features_scaled)
int(label)

5

In [37]:
# including the "clusters", "id" and "duration_ms" columns to the scaled_songs_df
scaled_songs_df["clusters"] = clusters
scaled_songs_df = scaled_songs_df.join(songs_df[["id", "duration_ms"]])
scaled_songs_df

,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,clusters,id,duration_ms
0,-1.164618,0.330871,-0.128308,-0.114356,-0.906894,-0.317436,-0.967829,1.556821,6,3q9G9SvgdfE1fxlvoDIb20,167720
1,-1.742480,1.487556,0.979190,1.318250,-0.899905,-0.517388,-1.484729,0.775615,1,47FyQCd3TYLrZ9TU6MPaWK,166707
2,-1.835872,1.015054,0.721642,-0.343092,-0.904931,0.365159,-1.360362,-0.278934,1,6p8eFfPw3nQkf37aT3AkmK,438120
3,-0.989508,1.162475,1.078280,0.331076,0.138198,0.558217,-0.400405,0.682918,1,2b9lp5A6CqSzwOrBfAFhof,296200
4,-1.182129,1.256975,0.740435,-0.285306,-0.852200,0.585796,-0.905646,-0.631976,1,1s4Ie0cT6P73SRSfh3oyGW,291373
...,...,...,...,...,...,...,...,...,...,...,...
1038,1.000908,0.062490,0.789767,-0.491168,-0.627176,-0.691140,0.334137,-1.293131,7,7jLbTp3qZzah9kMIdW8e5M,238053
1039,0.919190,-0.084931,0.385079,2.233191,0.341445,-0.248487,0.761649,-1.157136,0,2g8HN35AnVGIk7B8yMucww,252747
1040,0.732406,0.062490,0.650528,3.846378,-0.711959,-0.503598,-0.027304,1.877484,0,36UjTTrSwaAqLkAlf2Ooc4,230493
1041,0.633177,0.792033,0.546313,0.150495,-0.381536,0.965016,0.730557,-0.107890,4,2bLqfJjuC5syrsgDsZfGmn,179120


In [38]:
# including the "id" and "duration_ms" columns to the scaled_user_song_df
scaled_user_song_df = pd.DataFrame(user_song_audio_features_scaled, columns=["danceability", "energy", "loudness", "speechiness", "acousticness", "liveness", "valence", "tempo"])
scaled_user_song_df = scaled_user_song_df.join(user_song_df[["id", "duration_ms"]])
scaled_user_song_df.head()

,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,id,duration_ms
0,-0.271557,-0.466713,0.578347,-0.654894,1.270578,-0.583579,-0.688004,-1.387944,1JTQG8J8jN8tjDTyUWnaRe,186459


In [ ]:
# todo: use the "label" variable to pick a random song_id from the scaled_song_df by matching it with the "clusters" column